In [1]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from PIL import Image

import torch
import yaml

import os
import random
from ultralytics.utils.plotting import plot_labels
import pandas as pd
import sys

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
module_path = module_path+'/data_preprocessing'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

#import visualization_utils as visutils

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Setup complete ✅ (32 CPUs, 251.6 GB RAM, 0.5/125.8 GB disk)


/vast/palmer/home.grace/eec42/BirdDetector/src
/vast/palmer/home.grace/eec42/BirdDetector/src/data_preprocessing


In [2]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

0


In [3]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
pretrained_model_name = 'pfeifer_penguins_poland_10percentbckgd_yolov8m_120epoch'
model_path = 'runs/detect/' + pretrained_model_name + '/weights/best.pt'
model = YOLO(model_path, 'deepcoral_detect')
#model = YOLO("yolov8-modified.yaml").load("yolov8m.pt")
#model.load_weights('yolov8m.pt')


#PRETRAINED_MODEL_NAME = 'pfeifer_penguins_poland_10percentbckgd_yolov8m_120epoch'
#PRETRAINED_MODEL_PATH = 'src/model/runs/detect/' + PRETRAINED_MODEL_NAME + '/weights/best.pt'

#MODEL_NAME = 'pfeifer_penguins_poland_palmyra_10percent_bckgd_yolov8m_120epochs'
#TASK = 'detect' # Choose between: 'deepcoral_detect' 'detect'
#model = YOLO('yolov8m.pt')

In [4]:
model.task

'deepcoral_detect'

In [5]:
model.model.model
for i, a in enumerate(model.model.model):
    if i == 9:
        print("block", i)
        print(a)

block 9
SPPF(
  (cv1): Conv(
    (conv): Conv2d(576, 288, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(288, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(1152, 576, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(576, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (m): MaxPool2d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
)


In [6]:
# TRAIN the model on our dataset (data.yml config file) 

model_name = 'val_plots_TEST'

results = model.train(
   data='data.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=3,
   batch=16, #32,
   #cos_lr=True,
   #dropout=0.3,
   #optimizer='Adam',
   patience=50,
   device=0,
   verbose=True,
   val = True,
   #lr0=0.001,
   #lrf=0.0001,
   degrees=90, fliplr=0.5, flipud=0.5, scale=0.5, # augmentation parameters
   name=model_name)

New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
engine/trainer: task=deepcoral_detect, mode=train, model=runs/detect/pfeifer_penguins_poland_10percentbckgd_yolov8m_120epoch/weights/best.pt, data=data.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=val_plots_TEST5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_st

self.training True


AMP: checks passed ✅
train: Scanning /gpfs/gibbs/project/jetz/eec42/data/deepcoral_palmyraT__10percent_background/train/source/labels.cache... 3373 images, 237 backgrounds, 0 corrupt: 100%|██████████| 3610/3610 [00:00<?, ?it/s]
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/deepcoral_palmyraT__10percent_background/train/source/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_0.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/deepcoral_palmyraT__10percent_background/train/source/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_2.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/deepcoral_palmyraT__10percent_background/train/source/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_2.0_0.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec

METRICS IN DATALOADERS {'metrics/precision(B)': 0, 'metrics/recall(B)': 0, 'metrics/mAP50(B)': 0, 'metrics/mAP50-95(B)': 0, 'val/box_loss': 0, 'val/cls_loss': 0, 'val/dfl_loss': 0, 'val/coral_loss': 0}


Plotting labels to runs/deepcoral_detect/val_plots_TEST5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/deepcoral_detect/val_plots_TEST5
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss coral_loss  Instances       Size
        1/3      12.8G      4.046      5.144       2.85  9.406e-05        502        640:   0%|          | 0/76 [00:29<?, ?it/s]                   all        192       3819      0.736      0.632      0.631      0.257
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/6 [00:03<?, ?it/s]
        1/3      12.8G      4.046      5.144       2.85  9.406e-05

In [6]:
results

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x14d3fe0ab310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

## Visualization

In [11]:
TASK = 'deepcoral_detect' #detect
#model_name = 'deepcoral_test4'
#model_path = 'runs/' + TASK + '/' + model_name + '/weights/best.pt'
#model = YOLO(model_path, TASK)
model_name = 'deepcoral_background_lscale16_epochs20_coralgain10'
PRETRAINED_MODEL_PATH = 'runs/' + TASK + '/' + model_name + '/weights/best.pt'

model = YOLO(PRETRAINED_MODEL_PATH, TASK)

In [12]:
datasets = ['source', 'target'] #['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins', 'global_birds_pfeifer']
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'

In [9]:
# FOR NORMAL MODEL
# Select randomly 10 images from the test dataset

if TASK == 'detect':
    selected_img = []
    for subdataset in datasets:
        selected_img.extend(random.choices(os.listdir(img_path + subdataset + '/images/'), k=6))

    results = model.predict(
            #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
            source = [os.path.join(img_path + 'images/', img) for img in selected_img],
            conf = 0.2, 
            iou = 0.1,
            show=False,
            save=False
        )

In [10]:
if TASK == 'detect':
    for img, result in zip(selected_img, results):

        detection_boxes = []
        save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
        for detect in range(len(result.boxes.cls)):
            det = {}
            det['conf'] = result.boxes.conf[detect].cpu()
            det['category'] = result.boxes.cls[detect].cpu()
            coords = result.boxes.xywhn[detect].cpu()
            det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
            detection_boxes.append(det)
            
        im_path = os.path.join(img_path + 'images/', img)
        visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                        confidence_threshold=0.0, detector_label_map=None,
                                        thickness=1,expansion=0, colormap=['Red'])

        selected_label = img_path + 'labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
        if os.path.exists(selected_label):
            detection_boxes = []
            df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
            for irow, row in df.iterrows():  
                det = {}
                det['conf'] = None
                det['category'] = row[0]
                det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
                detection_boxes.append(det)
        
            # Draw annotations
            save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
            visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                            confidence_threshold=0.0, detector_label_map=None,
                                            thickness=1,expansion=0, colormap=['SpringGreen'])
                                            
            # Remove predictions-only images
            os.remove(save_path)


In [15]:
# FOR DEEP CORAL MODEL
# Select randomly 10 images from the test dataset

if TASK == 'deepcoral_detect':

    for subdataset in datasets:
        selected_img = random.choices(os.listdir(img_path + subdataset + '/images/'), k=12)

        results = model.predict(
                #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
                source = [os.path.join(img_path, subdataset + '/images/', img) for img in selected_img],
                conf = 0.2, 
                iou = 0.1,
                show=False,
                save=False
            )
        
        for img, result in zip(selected_img, results):

            detection_boxes = []
            save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/' + TASK + '/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
            for detect in range(len(result.boxes.cls)):
                det = {}
                det['conf'] = result.boxes.conf[detect].cpu()
                det['category'] = result.boxes.cls[detect].cpu()
                coords = result.boxes.xywhn[detect].cpu()
                det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
                detection_boxes.append(det)
                
            im_path = os.path.join(img_path + subdataset + '/images/', img)
            visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                            confidence_threshold=0.0, detector_label_map=None,
                                            thickness=1,expansion=0, colormap=['Red'])

            selected_label = img_path  + subdataset + '/labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
            if os.path.exists(selected_label):
                detection_boxes = []
                df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
                for irow, row in df.iterrows():  
                    det = {}
                    det['conf'] = None
                    det['category'] = row[0]
                    det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
                    detection_boxes.append(det)
        
                # Draw annotations
                save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/' + TASK + '/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
                visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                                confidence_threshold=0.0, detector_label_map=None,
                                                thickness=1,expansion=0, colormap=['SpringGreen'])
                
                # Remove predictions-only images
                os.remove(save_path)



0: 640x640 41 birds, 1: 640x640 26 birds, 2: 640x640 50 birds, 3: 640x640 5 birds, 4: 640x640 16 birds, 5: 640x640 3 birds, 6: 640x640 9 birds, 7: 640x640 3 birds, 8: 640x640 13 birds, 9: 640x640 54 birds, 10: 640x640 2 birds, 11: 640x640 8 birds, 58.0ms
Speed: 3.6ms preprocess, 4.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 57.6ms
Speed: 1.4ms preprocess, 4.8ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
b

NameError: name 'b' is not defined

## YOLO Evaluation

In [ ]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True, iou=0.1, conf=0.2, max_det=600)

# Export the model to ONNX format
#success = model.export(format='onnx')

In [ ]:
print(metrics.box.map)
print(metrics.box.map50)    # map50-95

## YOLO Evaluation per dataset

In [ ]:
datasets =  ['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins',
                    'global_birds_mckellar', 'global_birds_newmexico', 
                    'global_birds_pfeifer', 'uav_thermal_waterfowl']

In [ ]:
model_name = 'pfeifer_penguins_poland_palmyra_mckellar_yolov8m_120epoch'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

In [ ]:
for dataset in datasets:

    # Change test folder
    fname = "data.yaml"
    stream = open(fname, 'r')
    data = yaml.safe_load(stream)
    data['test'] = 'test/' + dataset + '/images/'
    with open(fname, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data, default_flow_style=False))
    
    metrics = model.val(split='test', save_json=True, iou=0.1, max_det=600)
    print(metrics.box.map50)

# Change test folder to original name
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
data['test'] = 'test/images/'

## **TEST**

In [ ]:
model_name = 'pfeifer_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience50_lr00.001_lrf0.0001'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

In [ ]:
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'

selected_img = (random.choices(os.listdir(img_path + '/images/'), k=1))
selected_img = [os.path.join(img_path + '/images/', x) for x in selected_img]
selected_img

['/gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer/test//images/global_birds_pfeifer_Fregata_Island_2016_Chinstrap_penguins_359_patch_0.0_0.0_448_448.jpg']

In [ ]:
output = model(selected_img)


0: 640x640 3 birds, 9.5ms
Speed: 3.8ms preprocess, 9.5ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 birds, 9.5ms
Speed: 3.3ms preprocess, 9.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
output

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bird'}
 orig_img: array([[[ 54,  60,  59],
         [ 59,  65,  64],
         [ 61,  66,  65],
         ...,
         [ 59,  63,  64],
         [ 57,  61,  62],
         [ 52,  56,  57]],
 
        [[ 49,  55,  54],
         [ 52,  58,  57],
         [ 58,  63,  62],
         ...,
         [ 60,  64,  65],
         [ 60,  64,  65],
         [ 56,  60,  61]],
 
        [[ 53,  59,  58],
         [ 52,  58,  57],
         [ 61,  66,  65],
         ...,
         [ 58,  62,  63],
         [ 61,  65,  66],
         [ 60,  64,  65]],
 
        ...,
 
        [[147, 154, 139],
         [147, 154, 139],
         [145, 152, 137],
         ...,
         [ 61,  62,  66],
         [ 66,  67,  71],
         [ 68,  69,  73]],
 
        [[141, 151, 135],
         [143, 153, 137],
         [143, 153, 137],
         ...,
         [ 53,  54,  58],
     